In [1]:
#import all dependencies
import gpt_2_simple as gpt2
from datetime import datetime
import numpy as np
import pickle
import sklearn
import pandas as pd
import re
import spacy
import keras
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model, Sequential
from keras.layers import Embedding, Dense, LSTM, Dropout, GlobalAveragePooling1D, Flatten, Bidirectional
from pickle import dump, load
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
#starting a gpt2 session with the fine-tuned model
sess = gpt2.start_tf_sess() 

In [3]:
# saver = tf.train.Saver()
# saver.restore(sess=sess, save_path='rn1')
gpt2.load_gpt2(sess, run_name='rn1') 

Loading checkpoint D:/quotes_project/checkpoint/rn1/model-17500
INFO:tensorflow:Restoring parameters from D:/quotes_project/checkpoint/rn1/model-17500


In [4]:
# while True: #I loop to allow the user to request quotes as many times as they want
quote_list = []
def load_quotes():
    global result
    samples = input("How many quotes do you wish to get?\n ")
    while True:
        try:
            samples = int(samples)
            break
        except ValueError:
            samples = input("invalid number, please enter a correct number\n: ")



    theme = input("Give a single word and get quotes about it: ")
    while " " in theme:
        theme = input("Please enter only a single word or remove any space:\n")
        if " " not in theme:
            break
    result = gpt2.generate(sess, run_name='rn1', nsamples= int(samples), length=50, 
                          temperature=0.7, truncate="<|endoftext|>",prefix= "<|startoftext|>"+theme, 
                          include_prefix=True, return_as_list=True) #Here the quote is generated with some parameters

    print("\n")
    for i in result:
      print('"'+i[15:]+'"'+'\n') #This part is only for evaluation purposes, so it can be removed
      quote_list.append(i[15:])

In [5]:
#This section takes the generated quotes, and passes it through the sentiment analysis model. Only the positive ones are
#returned.
#Note: The main purpose of this sentiment analysis is not really to determine if a quote is positive or negative, it is because
#since it is a text generator, some of the generated texts may be controversial or inappropriate, so this step will minimise the
#risk
def classify():
    with open('sentiment_analysis_tokenizer', 'rb') as handle:
        loaded_tokenizer = pickle.load(handle)
    model = tf.keras.models.load_model('sentiment_analysis_1', custom_objects={"CustomModel": Sequential})
    print('filtered: \n')
    for i in quote_list:
        seq = loaded_tokenizer.texts_to_sequences([i])
        sequences = np.array(seq)
        sequences = pad_sequences(sequences, maxlen= 161, dtype= 'int32', padding= 'pre', truncating= 'pre',  value=0.0)
        predict = model.predict(sequences)
        if np.argmax(predict) == 1:
          print(i) #Instead of printing, it can be used in so many ways

In [6]:
#This loop allows us to request as many quotes as we want. Keep in mind that the GPT-2 model is considerably slow in generating
#text. I am still trying to figure out a way to fasten it
while True:
    load_quotes()
    classify()
    quote_list = []
    print("\nWhant more?\n")
    more = input('Type "Yes" if you want to continue, or "No" if you want to stop: ')
    more = more.lower()
    options = {"no","nah","noh","nope","non"}
    if more in options:
        break

How many quotes do you wish to get?
 10
Give a single word and get quotes about it: thrive


"thrive, in this way, to attain, for real men are not sedentary, and, if they are ever bothered to walk, they are never sick."

"thrive to live in a way that honors what you love."

"thrive to be good in all things.If you're not, you're not real.And if you're not, you're not."

"thrive, succeed, and achieve your goals"

"thrive for higher thoughts"

"thrive to be courageous everyday, you will become the bravest and bravest thing in this world"

"thrive to be proactive instead of reactive"

"thrive for success,but never forget to work hard."

"thrive to live in a world of difference and contrast will manifest in abundance and joy"

"thrive to be happy and succeed"

filtered: 



c:\users\tsimalayh\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


thrive to live in a way that honors what you love.
thrive to be good in all things.If you're not, you're not real.And if you're not, you're not.
thrive, succeed, and achieve your goals
thrive to be courageous everyday, you will become the bravest and bravest thing in this world
thrive to be proactive instead of reactive
thrive to live in a world of difference and contrast will manifest in abundance and joy
thrive to be happy and succeed

Whant more?

Type "Yes" if you want to continue, or "No" if you want to stop: yes
How many quotes do you wish to get?
 10
Give a single word and get quotes about it: live


"live life in a unique way"

"live your life on a path that serves you.that is all there is."

"live in the moment, but not in the moment. live in the moment, but not in the moment. live in the moment, but not in the moment."

"live. Love. Breathe. Be. Love. Be."

"live life in complete bliss, peace and happiness."

"live a daring life."

"live your life with passion and purpose.You